In [ ]:
import numpy as np
import pandas as pd
import gspread
import itertools
import warnings
import pandas_gbq as pdq

Mounted at /content/drive


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import sklearn.ensemble
from sklearn.metrics import mean_squared_error, mean_absolute_error
#! pip install rdkit-pypi
#! pip install git+https://github.com/samoturk/mol2vec.git
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, PandasTools
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.Draw import IPythonConsole

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 59.8 MB/s eta 0:00:00
  Cloning https://github.com/samoturk/mol2vec.git to /tmp/pip-req-build-rpgpssos
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec.git /tmp/pip-req-build-rpgpssos
  Resolved https://github.com/samoturk/mol2vec.git to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14017 sha256=64cd8801498ac240a9990d900c5ce685888f8f0bf07eeca1a6bcd36a3d6291ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-3goendxr/wheels/f2/dc/10/ba71e3bd9749745fbcd888c99557b3c148247c1d98983d75f3
Successfully built mol2vec


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [ ]:
import mol2vec
from mol2vec import features
from mol2vec import helpers
from mol2vec.features import mol2alt_sentence

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#MAPK14



In [ ]:
#mapk14_1M
mapk14_1M_df = pd.read_parquet('data/mapk14_1M.parquet', engine='pyarrow')
mapk14_1M_df = mapk14_1M_df[['smiles','target_enrichment']]
mapk14_1M_df.head(2)

,smiles,target_enrichment
0,CCCCCC/C=C/C(=O)N(CC(=O)NCc1cc(C(=O)NC)c(C)o1)...,0.321008
1,CNC(=O)[C@@H](CCNC(=O)[C@@H](CCNC(=O)c1ccc2ccc...,0.367196


In [ ]:
mapk14_1M_df.iloc[0]['smiles']

'CCCCCC/C=C/C(=O)N(CC(=O)NCc1cc(C(=O)NC)c(C)o1)Cc1ccccc1'

In [ ]:
len(mapk14_1M_df)

1000000

In [ ]:
mapk14_1M_df.describe()

,target_enrichment
count,1000000.000000
mean,0.525482
std,0.281425
min,0.315125
25%,0.395197
50%,0.440103
75%,0.596901
max,22.056560


In [ ]:
mapk14_1M_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   smiles             1000000 non-null  string 
 1   target_enrichment  1000000 non-null  float64
dtypes: float64(1), string(1)
memory usage: 15.3 MB


In [ ]:
#Turn SMILES into mol
def smiles_to_mol2vec(smiles):
    #Convert SMILES to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")

    #Convert RDKit molecule to a Mol2Vec vector
    mol2vec_vector = mol2alt_sentence(mol,2)

    return mol2vec_vector

In [ ]:
# Apply smiles_to_mol2vec to the 'smiles' column and create a new column 'mol2vec'
# This cell takes a couple of minutes to run
mapk14_1M_df['mol2vec'] = mapk14_1M_df['smiles'].apply(smiles_to_mol2vec)
mapk14_1M_df

In [ ]:
# Save the output from previous cell
# mapk14_1M_df.to_csv('mapk14_1M_df.csv')

In [ ]:
# Use if the dataframe from the previous cell is saved/uploaded
# mapk14_1M_df = pd.read_csv('/content/mapk14_1M_df.csv', index_col=[0])

##Splits

In [ ]:
#mapk14_random
mapk14_random_df = pd.read_parquet('data/mapk14_random.parquet', engine='pyarrow')
#mapk14_disynthon
mapk14_disynthon_df = pd.read_parquet('data/mapk14_disynthon.parquet', engine='pyarrow')

,split1,split2,split3,split4,split5
0,valid,train,train,valid,train
1,train,valid,train,train,train


##Model


In [ ]:
mapk14_model_df = mapk14_1M_df[['smiles','mol2vec','target_enrichment']]
mapk14_model_df.head(2)

,smiles,mol2vec,target_enrichment
0,CCCCCC/C=C/C(=O)N(CC(=O)NCc1cc(C(=O)NC)c(C)o1)...,"['2246728737', '3542456614', '2245384272', '11...",0.321008
1,CNC(=O)[C@@H](CCNC(=O)[C@@H](CCNC(=O)c1ccc2ccc...,"['2246728737', '3824063894', '847961216', '828...",0.367196


In [ ]:
# Truncate mol2vec vector
# This cell takes a couple of minutes to run

import ast

def truncate_mol2vec(mol2vec_str):
  try:
    mol2vec_list = ast.literal_eval(mol2vec_str)
    return mol2vec_list[:30]  # Truncate to the first 30 elements
  except (SyntaxError, ValueError):
    return None

#Apply the function to the 'mol2vec' column
mapk14_model_df['mol2vec_truncated'] = mapk14_model_df['mol2vec'].apply(truncate_mol2vec)

<ipython-input-73-83bc2d712f81>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapk14_model_df['mol2vec_truncated'] = mapk14_model_df['mol2vec'].apply(truncate_mol2vec)


In [ ]:
y_test_list = []
y_pred_list = []

for i in range(1,6):
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #We already have this
  train_df = mapk14_model_df[mapk14_random_df['split'+str(i)]=='train']
  valid_df = mapk14_model_df[mapk14_random_df['split'+str(i)]=='valid']
  test_df = mapk14_model_df[mapk14_random_df['split'+str(i)]=='test']

  X_train = np.array(train_df['mol2vec_truncated'].tolist(), dtype=object)
  X_test = np.array(test_df['mol2vec_truncated'].tolist(), dtype=object)
  y_train = train_df['target_enrichment']
  y_test = test_df['target_enrichment']

  #Train an XGBoost model
  model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
  model.fit(X_train, y_train)

  #Make predictions
  y_pred = model.predict(X_test)
  y_pred_list.append(y_pred)
  y_test_list.append(y_test)

In [ ]:
mse_list = []
for y_test, y_pred in zip(y_test_list, y_pred_list):
  mse = mean_squared_error(y_test, y_pred)
  mse_list.append(mse)

average_mse = np.mean(mse_list)
print(f'Average Mean Squared Error: {average_mse}')

Average Mean Squared Error: 0.08500918154082612


##Results for held-out test set

Pending model evaluation

In [ ]:
mapk14_ondna_df = pd.read_csv('data/mapk14_ondna.csv')
mapk14_ondna_df.head(2)

,Unnamed: 0,smiles,molecule_hash,kd,smiles_a,smiles_b,smiles_c
0,0,Nc1n(c2ccccc2)ncc1C(NCc3ccc(C(NC(C(NC)=O)C(OC(...,NaN,384.0,NaN,NaN,NaN
1,1,Nc1n(c2ccc(F)cc2)ncc1C(NCC3=CC=C(C=C3)C(NC(C(N...,NaN,397.9,NaN,NaN,NaN


In [ ]:
mapk14_offdna_df = pd.read_csv('data/mapk14_offdna.csv')
mapk14_offdna_df.head(2)

,Unnamed: 0,smiles,molecule_hash,kd,smiles_a,smiles_b,smiles_c
0,0,CC1=CC=C(C(NC2=CC=C(CN3CCN(C)CC3)C(C(F)(F)F)=C...,NaN,10.3,NaN,NaN,NaN
1,1,CCCC1CCC(C(=O)NC2CCN(CC(=O)N3CCC[C@@H](C(=O)NC...,4010bb979f72433a7c4dd6a1866281ae52ca26799b4eb2...,64900.0,CNC(=O)[C@@H]1CCCNC1,CCCC1CCC(CC1)C(O)=O,OC(=O)CN1CCC(CC1)NC(=O)OCC1c2ccccc2-c2ccccc12


#DDR1


In [ ]:
#ddr1_1M
ddr1_1M_df = pd.read_parquet('data/ddr1_1M.parquet', engine='pyarrow')
ddr1_1M_df = ddr1_1M_df[['smiles','target_enrichment']]
ddr1_1M_df.head(2)

,smiles,target_enrichment
0,CNC(=O)[C@H](CNC(=O)c1ccccc1CNC(=O)c1ccc(-c2no...,0.352696
1,CNC(=O)[C@H](CCc1ccccc1)NC(=O)CN1C(=O)[C@@H](N...,1.482711


In [ ]:
len(ddr1_1M_df)

1000000

In [ ]:
ddr1_1M_df.describe()

,target_enrichment
count,1000000.000000
mean,0.657953
std,0.893551
min,0.291384
25%,0.295446
50%,0.420473
75%,0.619895
max,62.429077


In [ ]:
ddr1_1M_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   smiles             1000000 non-null  string 
 1   target_enrichment  1000000 non-null  float64
dtypes: float64(1), string(1)
memory usage: 15.3 MB


##Splits

In [ ]:
#ddr1_random
ddr1_random_df = pd.read_parquet('data/ddr1_random.parquet', engine='pyarrow')
#ddr1_disynthon
ddr1_disynthon_df = pd.read_parquet('data/ddr1_disynthon.parquet', engine='pyarrow')

##Model

In [ ]:
# Apply smiles_to_mol2vec to the 'smiles' column and create a new column 'mol2vec'
ddr1_1M_df['mol2vec'] = ddr1_1M_df['smiles'].apply(smiles_to_mol2vec)
ddr1_1M_df.head(2)

In [ ]:
ddr1_model_df = ddr1_1M_df[['smiles','mol2vec','target_enrichment']]
ddr1_model_df.head(2)

In [ ]:
# Apply the function to the 'mol2vec' column
ddr1_model_df['mol2vec_truncated'] = ddr1_model_df['mol2vec'].apply(truncate_mol2vec)

In [ ]:
y_test_list = []
y_pred_list = []

for i in range(1,6):
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #We already have this
  train_df = ddr1_model_df[ddr1_random_df['split'+str(i)]=='train']
  valid_df = ddr1_model_df[ddr1_random_df['split'+str(i)]=='valid']
  test_df = ddr1_model_df[ddr1_random_df['split'+str(i)]=='test']

  X_train = np.array(train_df['mol2vec_truncated'].tolist(), dtype=object)
  X_test = np.array(test_df['mol2vec_truncated'].tolist(), dtype=object)
  y_train = train_df['target_enrichment']
  y_test = test_df['target_enrichment']

  # Train an XGBoost model
  model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
  model.fit(X_train, y_train)

  # Make predictions
  y_pred = model.predict(X_test)
  y_pred_list.append(y_pred)
  y_test_list.append(y_test)

In [ ]:
mse_list = []
for y_test, y_pred in zip(y_test_list, y_pred_list):
  mse = mean_squared_error(y_test, y_pred)
  mse_list.append(mse)

average_mse = np.mean(mse_list)
print(f'Average Mean Squared Error: {average_mse}')

##Results for held-out test set

Pending model evaluation

In [ ]:
ddr1_ondna_df = pd.read_csv('data/ddr1_ondna.csv')
ddr1_ondna_df.head(2)

,Unnamed: 0,smiles,molecule_hash,kd,smiles_a,smiles_b,smiles_c
0,0,CNC(CN1CN(C2(C1=O)CCN(C(c(cn3)cc4c3[nH]nc4)=O)...,NaN,70.6,NaN,NaN,NaN
1,1,Nc1n(c2ccccc2)ncc1C(NCc3ccc(C(N[C@@H](CCC4CCCC...,NaN,41.4,NaN,NaN,NaN


In [ ]:
ddr1_offdna_df = pd.read_csv('data/ddr1_offdna.csv')
ddr1_offdna_df.head(2)

,Unnamed: 0,smiles,molecule_hash,kd,smiles_a,smiles_b,smiles_c
0,0,CC1=CC=C(C(NC2=CC=C(CN3CCN(C)CC3)C(C(F)(F)F)=C...,NaN,0.0,NaN,NaN,NaN
1,1,CC(C1=CN(CC(NC2=CC=C(NC3=NC=NC4=CC(OCC(O)=O)=C...,NaN,64100.0,NaN,NaN,NaN
